In [1]:
import codecs
import jieba
import jieba.posseg as pseg
import jieba.analyse
import paddle
import re

In [2]:
paddle.enable_static()
jieba.enable_paddle()
jieba.load_userdict('../resource/dict.txt.big')
jieba.analyse.set_stop_words("../resource/stopwords.txt")

Paddle enabled successfully......
Building prefix dict from the default dictionary ...
Dumping model to file cache /var/folders/q9/0yjtkz2x1mq4f7qlvql797l80000gn/T/jieba.cache
Loading model cost 1.133 seconds.
Prefix dict has been built successfully.


In [3]:
names = {}
relationships = {}
lineNames = []

In [4]:
with open('../resource/stopwords.txt', 'r', encoding='utf8') as f:
    stops = f.read().split('\n')

In [5]:
text=open("../1.txt", encoding='utf-8').read()
#text=text.replace('\n','')

In [6]:
punc = "[【】╮╯▽╰╭★→「」！，❤。～《》：()／（）”“；：、！：，。？、~@#￥%……&*]+"
token = " ".join(jieba.cut(text,cut_all=False,use_paddle=True))
# 移除停用字
token = " ".join([term for term in token.split() if term not in stops])
# 移除標點符號
token = re.sub(punc, "", token)
poss = pseg.cut(token,use_paddle=True)        # 分词并返回该词词性
lineNames.append([])        # 为新读入的一段添加人物名称列表
for w in poss:
    if w.flag != "nr" and "nrt" or len(w.word)<2:
        #w.flag != "nr" or  w.flag != "nrt" or len(w.word)<2 
        continue            # 当分词长度小于2或该词词性不为nr时认为该词不为人名
    lineNames[-1].append(w.word)        # 为当前段的环境增加一个人物
    if names.get(w.word) is None:
        names[w.word] = 0
        relationships[w.word] = {}
    names[w.word] += 1                    # 该人物出现次数加 1

In [7]:
for line in lineNames:                    # 对于每一段
    for name1 in line:                    
        for name2 in line:                # 每段中的任意两个人
            if name1 == name2:
                continue
            if relationships[name1].get(name2) is None:        # 若两人尚未同时出现则新建项
                relationships[name1][name2]= 1
            else:
                relationships[name1][name2] = relationships[name1][name2]+ 1        # 两人共同出现次数加 1

In [8]:
with codecs.open("node.txt", "w", "utf8") as f:
    f.write("Id Label Weight\r\n")
    for name, times in names.items():
        f.write(name + " " + name + " " + str(times) + "\r\n")
 
with codecs.open("edge.txt", "w", "utf8") as f:
    f.write("Source Target Weight\r\n")
    for name, edges in relationships.items():
        for v, w in edges.items():
            if w > 3:
                f.write(name + " " + v + " " + str(w) + "\r\n")